## 巡回セールスマン問題をQAOAで解く

### 問題設定
- 3つの都市 A, B, C がある
- 移動距離の総和が最小となるような都市の巡回順序 (0,1,2) を考える
- 同じ都市は 2 回訪問しない
- 1 回の訪問につき 1 つの都市を訪問する
- スタートとゴールは同じ都市とする（巡回問題）
- 各都市間の距離は下表により定める（例: 都市 A から都市 B に移動するときの距離は 2 である）
- 以降のコスト関数の定式化では、この表に示された都市間距離 $d_{ij}$ を使用する  
  (例: $d_{0,1}=2$ は都市A→都市Bへの移動距離が2であることを表す)


|       | A  | B | C  |
| ----- | -- | - | -- |
| **A** | 0  | 2 | 9  |
| **B** | 1  | 0 | 6  |
| **C** | 15 | 7 | 0  |

### 二値変数の設定
4都市 $A, B, C$ を順に 0, 1, 2 と番号付けする  
二値変数 $q_{i,j} \in \{0,1\}$ を、
「巡回順序 $i$ に都市 $j$ を訪れる」場合に1をとる変数と定義する

下表は、縦軸を巡回順序 $i$、横軸を都市 $j$ として、二値変数の配置を示す

|       | A      | B      | C      |
| ----- | ------ | ------ | ------ |
| **0** |$q_{0,0}$|$q_{0,1}$|$q_{0,2}$|
| **1** |$q_{1,0}$|$q_{1,1}$|$q_{1,2}$|
| **2** |$q_{2,0}$|$q_{2,1}$|$q_{2,2}$|

### コスト関数
訪問距離 $d$ は、順序 $i$ から次の順序 $(i+1)\;\bmod\;3$ へ移動したときの都市間距離の総和である  
ここで $q_{i,j}\in\{0,1\}$ は「順序 $i$ に都市 $j$ を訪問する」ことを表す二値変数である  

$$
d = \sum_{i=0}^{2}\sum_{j=0}^{2}\sum_{k=0}^{2} d_{j,k} \ast q_{i,j} \ast q_{(i+1)\bmod 3, k}
$$

### コスト関数のハミルトニアン
$q = \frac{1 - z}{2}$の関係により二値変数 $q$ を演算子 $Z$ に置き換えると、コスト関数のハミルトニアン $\hat{H}_d$ は以下のように定式化される

$$
\begin{align*}
\hat{H}_d
  &= \frac{1}{4} \sum_{i=0}^{2}\sum_{j=0}^{2}\sum_{k=0}^{2} d_{j,k} \ast (
  - Z_{i,j} - Z_{(i+1) \bmod 3,k} + Z_{i,j}\;Z_{(i+1) \bmod 3,k}
  ) + const
\end{align*}
$$

### 制約条件

1. **順番(行)方向の制約**（一回の訪問で訪れる都市は1つだけ）：
$$
\sum_{j=0}^{2} q_{i,j} = 1, \quad \forall i = 0,1,2,3
$$

2. **都市方向(列)の制約**（各都市は一度だけ訪問される）：
$$
\sum_{i=0}^{2} q_{i,j} = 1, \quad \forall j = 0,1,2,3
$$

### 制約条件のハミルトニアン
1. **順番(行)方向のハミルトニアン**

$$
\begin{align*}
     \hat{H}_{row} 
     &= \sum_{i=0}^{2}
     \left(
         \sum_{j=0}^{2} q_{i,j} - 1
     \right)^2
\end{align*}
$$

2. **都市(列)方向のハミルトニアン**
$$
\begin{align*}
     \hat{H}_{col} 
     &= \sum_{j=0}^{2}
     \left(
         \sum_{i=0}^{2} q_{i,j} - 1
     \right)^2
\end{align*}
$$

$q = \frac{1 - z}{2}$の関係により二値変数 $q$ を演算子 $Z$ に置き換えると、コスト関数のハミルトニアン $\hat{H}_d$ は以下のように定式化される
$$
\hat{H}_{row} = \frac{1}{2}
    \left (
       - \sum_{i=0}^{2}\sum_{j=0}^{2} Z_{i,j} + \sum_{i=0}^{2}\sum_{j=0}^{2}\sum_{k=j+1}^{2} Z_{i,j}Z_{i,k}
    \right) + const
$$
$$
\hat{H}_{col} = \frac{1}{2}
    \left (
       - \sum_{i=0}^{2}\sum_{j=0}^{2} Z_{i,j} + \sum_{j=0}^{2}\sum_{i=0}^{2}\sum_{k=i+1}^{2} Z_{i,j}Z_{k,j}
    \right) + const
$$


## 時間発展演算子
$A \in \mathbb{R}$を制約条件のハミルトニアンの重み、$\gamma \in \mathbb{R}$を最適化するハイパーパラメータとする    
全体のハミルトニアン $\hat{H}$ を $\hat{H}=\hat{H}_d + A(\hat{H}_{row}+\hat{H}_{col})$により定めると、
時間発展演算子 $U$ は、

\begin{align*}
U &= \exp\!\left( -i\gamma \hat{H} \right) \\
  &= \exp\!\left\lbrack
  -i\gamma \left(
      \hat{H}_d + A\hat{H}_{row}+A\hat{H}_{col}
      \right) 
  \right \rbrack) \\
\end{align*}

となる。トロッター分解の近似を考えればこれは下式のように変形できる。
\begin{align*}
U \rightarrow \exp\!\left\lbrack
  -i\gamma \hat{H}_d
  \right \rbrack
  \exp\!\left\lbrack
  -i\gamma A\hat{H}_{row}
  \right \rbrack
    \exp\!\left\lbrack
  -i\gamma A\hat{H}_{col}
  \right \rbrack
  \\
\end{align*}

### Qiskitによる実装

In [1]:
import numpy as np
from qiskit.circuit import QuantumCircuit, Parameter

In [2]:
# 諸条件

# 量子ビット数
n_qubit = 9

# 繰り返し回数
p = 2

# 距離行列
d = np.array([
    [0,2,9],
    [1,0,6],
    [15,7,0],
])

In [3]:
# 制約条件の重み
A = Parameter("A")

# 最適化パラメータ
gamma = [Parameter(f"γ{n}") for n in range(p)]
beta = [Parameter(f"β{n}") for n in range(p)]
print(gamma)
print(beta)

[Parameter(γ0), Parameter(γ1)]
[Parameter(β0), Parameter(β1)]


In [4]:
# コスト関数ハミルトニアンの実装
def cost_hamiltonian_gate(gamma):
    
    qc = QuantumCircuit(n_qubit, name="cost_hamiltonian")
    qc = QuantumCircuit(n_qubit, name="cost_hamiltonian")    
    for i in range(3):
        for j in range(3):
            for k in range(3):
                qc.rz(-0.5*gamma*d[j][k], 3*i+j)
    
    for i in range(3):
        for j in range(3):
            for k in range(3):
                qc.rz(-0.5*gamma*d[j][k], 3*((i+1)%3)+k)
    
    for i in range(3):
        for j in range(3):
            for k in range(3):
                qc.rzz(+0.5*gamma*d[j][k], 3*i+j, 3*((i+1)%3)+k)

    return qc.to_gate()

In [5]:
def row_constraint_hamiltonian_gate(A, gamma):
    
    qc = QuantumCircuit(n_qubit, name="row_constraint_hamiltonian")

    for i in range(3):
        for j in range(3):
            qc.rz(-A*gamma, 3*i+j)

    for i in range(3):
        for j in range(3):
            for k in range(j+1, 3):
                qc.rzz(A*gamma, 3*i+j, 3*i+k)

    return qc.to_gate()

In [6]:
def col_constraint_hamiltonian_gate(A, gamma):
    
    qc = QuantumCircuit(n_qubit, name="col_constraint_hamiltonian")

    for i in range(3):
        for j in range(3):
            qc.rz(-A*gamma, 3*i+j)

    for j in range(3):
        for i in range(3):
            for k in range(i+1, 3):
                qc.rzz(A*gamma, 3*i+j, 3*k+j)

    return qc.to_gate()